<a href="https://colab.research.google.com/github/vignesh99/CS6300-Speech-Technology/blob/master/HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
                                    #Get data files onto root
%cd
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/1 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/2 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/3 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/6 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/7 /root
!cp -avr /content/gdrive/My\ Drive/ctsHMM /root
!ls

/root
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
removed '/root/1'
'/content/gdrive/My Drive/HMM_IsoData/1' -> '/root/1'
removed '/root/2'
'/content/gdrive/My Drive/HMM_IsoData/2' -> '/root/2'
removed '/root/3'
'/content/gdrive/My Drive/HMM_IsoData/3' -> '/root/3'
removed '/root/6'
'/content/gdrive/My Drive/HMM_IsoData/6' -> '/root/6'
removed '/root/7'
'/content/gdrive/My Drive/HMM_IsoData/7' -> '/root/7'
'/content/gdrive/My Drive/ctsHMM/test/3.mfcc' -> '/root/ctsHMM/test/3.mfcc'
'/content/gdrive/My Drive/ctsHMM/test/2.mfcc' -> '/root/ctsHMM/test/2.mfcc'
'/content/gdrive/My Drive/ctsHMM/test/1.mfcc' -> '/root/ctsHMM/test/1.mfcc'
'/content/gdrive/My Drive/ctsHMM/test/5.mfcc' -> '/root/ctsHMM/test/5.mfcc'
'/content/gdrive/My Drive/ctsHMM/test/4.mfcc' -> '/root/ctsHMM/test/4.mfcc'
'/content/gdrive/My Drive/ctsHMM/dev/277.mfcc' -> '/root/ctsHMM/dev/277.mfcc'
'/content/gdrive/My Drive/ctsHMM/dev/61.mfcc'

In [89]:
!pip install pickle-mixin
from pickle import dump
from pickle import load

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-cp36-none-any.whl size=5998 sha256=84b4b31c88f9535f37c443c10e1c808430eaeaed4b35705452a2c1c24d78f53f
  Stored in directory: /root/.cache/pip/wheels/cd/05/42/71de70fa36b9cbb7657bb5793a16f8028c1cdc1bdd3b8e1ac3
Successfully built pickle-mixin


In [3]:
                                    #Import required libraries
from pylab import *
import os                           #Going through folders
from scipy.cluster.vq import kmeans2
from google.colab import files
from sklearn.cluster import KMeans

**Discete HMMs**

In [4]:
def fileTofeature(fileMfcc) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
                                    
    return Nf,features


In [6]:
                                    #Give the file with sequences for training data
def trainCentre(Nclust=20,folders=None) :
    if folders is None :            #Go through all digits
        folders = array(["1","2","3","6","7"])

    ind = []                     #All digits total vector numbers
    ind.append([0,0])            #Inital values
    Nwin = []                       #No. of vectors in each train file
    count = 0
    for word in folders :
                                    #Go through required folders
        for subdir, dirs, files in os.walk(word):
            dirname = subdir.split(os.path.sep)[-1]
            print('Directory:', dirname)
            if dirname == "train":
                for f in files:
                    if f.endswith('.mfcc'):
                                    #Copy file (mfcc) to root and perform K-means
                        !cp -avr /root/$word/train/$f /root
                        foo,temp = fileTofeature(f)
                                    #Concatenate training feature vectors
                        if count == 0 :
                            features = temp
                            Nwin.append(foo)
                        else :
                            features = np.concatenate((features,temp))
                            Nwin.append(foo+Nwin[-1])
                        !rm $f
                        count+=1
        ind.append([len(Nwin),Nwin[-1]])
                                    #Apply K-means on the vectors
    trainKM = KMeans(n_clusters=Nclust)
    trainKM.fit(features)
                                    #Get Nwin in required format
    ind = array(ind)
    Nwin = array(Nwin)
    for i in range(5) :
        Nwin[ind[i,0]:ind[i+1,0]] -=  ind[i,1]

    return Nwin,features,trainKM,ind


In [7]:
def seqDwnld(seq,fileWord) :
    with open(fileWord,"w") as f:
        f.write("\n".join(" ".join(map(str, line)) for line in seq))

    return None

In [8]:
                                    #Give the file with sequences for training data
def trainSeq(Nwin,features,trainKM,fileWord="check.txt") :
    seq = []
    count = 0
                                    #Go through required folders
    for i in range(0,len(Nwin)) :
                                    #Get each features from all-features-array
        if i != 0:
            temp = features[Nwin[i-1]:Nwin[i]]
        else : 
            temp = features[:Nwin[0]]
                                    #Perform Kmeans on predefined centres
        label = trainKM.predict(temp)
        seq.append(label)

        if i==2 :
            print(label)

    seq = array(seq)

    seqDwnld(seq,fileWord)

    return None


In [9]:
!ls

1  2  3  6  7  ctsHMM  ra_1.mfcc


In [10]:
Nw,feat,cenKM,ind = trainCentre()       #Train all the phenomes

                                    #Get file sequences
trainSeq(Nw[ind[0,0]:ind[1,0]],feat[ind[0,1]:ind[1,1]],cenKM,"one.seq")
trainSeq(Nw[ind[1,0]:ind[2,0]],feat[ind[1,1]:ind[2,1]],cenKM,"two.seq")
trainSeq(Nw[ind[2,0]:ind[3,0]],feat[ind[2,1]:ind[3,1]],cenKM,"three.seq")
trainSeq(Nw[ind[3,0]:ind[4,0]],feat[ind[3,1]:ind[4,1]],cenKM,"six.seq")
trainSeq(Nw[ind[4,0]:ind[5,0]],feat[ind[4,1]:ind[5,1]],cenKM,"seven.seq")


Directory: 1
Directory: train
'/root/1/train/ac_1.mfcc' -> '/root/ac_1.mfcc'
'/root/1/train/ag_1.mfcc' -> '/root/ag_1.mfcc'
'/root/1/train/ai_1.mfcc' -> '/root/ai_1.mfcc'
'/root/1/train/an_1.mfcc' -> '/root/an_1.mfcc'
'/root/1/train/bh_1.mfcc' -> '/root/bh_1.mfcc'
'/root/1/train/bi_1.mfcc' -> '/root/bi_1.mfcc'
'/root/1/train/br_1.mfcc' -> '/root/br_1.mfcc'
'/root/1/train/ca_1.mfcc' -> '/root/ca_1.mfcc'
'/root/1/train/cg_1.mfcc' -> '/root/cg_1.mfcc'
'/root/1/train/cl_1.mfcc' -> '/root/cl_1.mfcc'
'/root/1/train/cm_1.mfcc' -> '/root/cm_1.mfcc'
'/root/1/train/dc_1.mfcc' -> '/root/dc_1.mfcc'
'/root/1/train/dg_1.mfcc' -> '/root/dg_1.mfcc'
'/root/1/train/ea_1.mfcc' -> '/root/ea_1.mfcc'
'/root/1/train/ec_1.mfcc' -> '/root/ec_1.mfcc'
'/root/1/train/ee_1.mfcc' -> '/root/ee_1.mfcc'
'/root/1/train/eg_1.mfcc' -> '/root/eg_1.mfcc'
'/root/1/train/ei_1.mfcc' -> '/root/ei_1.mfcc'
'/root/1/train/ek_1.mfcc' -> '/root/ek_1.mfcc'
'/root/1/train/es_1.mfcc' -> '/root/es_1.mfcc'
'/root/1/train/hg_1.mfcc' -> '

In [15]:
                                    #Download the train sequence files
files.download("one.seq")
files.download("two.seq")
files.download("three.seq")
files.download("six.seq")
files.download("seven.seq")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
!ls
print(len(feat))
Nw

1  3  7       one.seq	 seven.seq  three.seq
2  6  ctsHMM  ra_1.mfcc  six.seq    two.seq
19968


array([  75,  159,  238,  322,  423,  520,  636,  730,  831,  925, 1028,
       1222, 1330, 1418, 1503, 1592, 1662, 1759, 1856, 1948, 2045, 2141,
       2238, 2359, 2444, 2538, 2646, 2721, 2818, 2910, 3034, 3146, 3224,
       3305, 3412, 3496, 3606, 3684, 3769,   70,  153,  233,  316,  422,
        525,  645,  743,  830,  937, 1038, 1126, 1205, 1289, 1372, 1444,
       1575, 1662, 1737, 1825, 1942, 2034, 2132, 2230, 2308, 2432, 2517,
       2601, 2677, 2770, 2876, 2969, 3068, 3153, 3256, 3343, 3456, 3534,
       3631,   71,  159,  242,  349,  452,  540,  646,  754,  843, 1024,
       1154, 1247, 1330, 1411, 1494, 1577, 1671, 1764, 1865, 1953, 2047,
       2159, 2275, 2458, 2538, 2621, 2733, 2829, 2930, 3024, 3148, 3254,
       3335, 3422, 3526, 3615, 3707, 3792, 3898,   97,  207,  341,  458,
        565,  696,  825,  946, 1061, 1169, 1309, 1435, 1548, 1645, 1761,
       1872, 1966, 2076, 2178, 2317, 2439, 2561, 2722, 2837, 2936, 3053,
       3161, 3269, 3377, 3490, 3635, 3760, 3876, 39

In [78]:
def testKmeans(fileMfcc,trainKM,Nclust=20,iters=1) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
    
                                    #Apply K-means on the vectors
    # if len(features) == Nf and len(features[0]) == Nc :
    #     label = trainKM.predict(features)
    label = trainKM.predict(features)
    
    return label


In [13]:
                                    #Give the file with sequences for testing data
def testSeq(trainKM,fileWord="check.txt",folders=None) :
    if folders is None :
        folders = array(["1","2","3","6","7"])
    seq = []
    count = np.zeros(5)
    for word in folders :    
                                        #Go through required folders
        for subdir, dirs, files in os.walk(word):
            dirname = subdir.split(os.path.sep)[-1]
            print('Directory:', dirname)
            if dirname == "dev":
                for f in files:
                    if f.endswith(".mfcc"):
                                        #Copy file (mfcc) to root and perform K-means
                        !cp -avr /root/$word/dev/$f /root
                        label = testKmeans(f,trainKM)
                        seq.append(label)
                        !rm $f
                        count[np.where(word == folders)[0][0]] += 1
        if word != "1" :
            count[np.where(word == folders)[0][0]] = count[np.where(word == folders)[0][0]] + count[np.where(word == folders)[0][0]-1]

    print(count)
    seq = array(seq)
    
    seqDwnld(seq,fileWord)

    return count


In [14]:
                                    #Get the test sequence
count = testSeq(cenKM,"DHMM.test")

Directory: 1
Directory: train
Directory: dev
'/root/1/dev/mk_1.mfcc' -> '/root/mk_1.mfcc'
'/root/1/dev/mm_1.mfcc' -> '/root/mm_1.mfcc'
'/root/1/dev/ms_1.mfcc' -> '/root/ms_1.mfcc'
'/root/1/dev/mw_1.mfcc' -> '/root/mw_1.mfcc'
'/root/1/dev/nc_1.mfcc' -> '/root/nc_1.mfcc'
'/root/1/dev/ng_1.mfcc' -> '/root/ng_1.mfcc'
'/root/1/dev/nh_1.mfcc' -> '/root/nh_1.mfcc'
'/root/1/dev/pe_1.mfcc' -> '/root/pe_1.mfcc'
'/root/1/dev/pk_1.mfcc' -> '/root/pk_1.mfcc'
'/root/1/dev/pm_1.mfcc' -> '/root/pm_1.mfcc'
'/root/1/dev/pp_1.mfcc' -> '/root/pp_1.mfcc'
'/root/1/dev/ra_1.mfcc' -> '/root/ra_1.mfcc'
Directory: 2
Directory: train
Directory: dev
'/root/2/dev/mk_2.mfcc' -> '/root/mk_2.mfcc'
'/root/2/dev/mm_2.mfcc' -> '/root/mm_2.mfcc'
'/root/2/dev/ms_2.mfcc' -> '/root/ms_2.mfcc'
'/root/2/dev/mw_2.mfcc' -> '/root/mw_2.mfcc'
'/root/2/dev/nc_2.mfcc' -> '/root/nc_2.mfcc'
'/root/2/dev/ng_2.mfcc' -> '/root/ng_2.mfcc'
'/root/2/dev/nh_2.mfcc' -> '/root/nh_2.mfcc'
'/root/2/dev/pe_2.mfcc' -> '/root/pe_2.mfcc'
'/root/2/d

In [16]:
                                    #Download the test sequence files
files.download("DHMM.test")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
                                    #Final probability files
!cp -avr /content/gdrive/My\ Drive/fp1.txt /root
!cp -avr /content/gdrive/My\ Drive/fp2.txt /root
!cp -avr /content/gdrive/My\ Drive/fp3.txt /root
!cp -avr /content/gdrive/My\ Drive/fp6.txt /root
!cp -avr /content/gdrive/My\ Drive/fp7.txt /root
!ls

'/content/gdrive/My Drive/fp1.txt' -> '/root/fp1.txt'
'/content/gdrive/My Drive/fp2.txt' -> '/root/fp2.txt'
'/content/gdrive/My Drive/fp3.txt' -> '/root/fp3.txt'
'/content/gdrive/My Drive/fp6.txt' -> '/root/fp6.txt'
'/content/gdrive/My Drive/fp7.txt' -> '/root/fp7.txt'
1  3  7       DHMM.test  fp2.txt  fp6.txt  one.seq    six.seq	 two.seq
2  6  ctsHMM  fp1.txt	 fp3.txt  fp7.txt  seven.seq  three.seq


In [22]:
def testProb(fileProb) :
    prob = np.genfromtxt(fileProb,delimiter=' ',dtype=str)
    prob = prob.astype(float)

    return prob

In [23]:
def accModel(count) :

    error = np.zeros(5)
    prob1 = testProb("fp1.txt")
    allprob = np.zeros((len(prob1),5))
    allprob[:,0] = prob1
    
    allprob[:,1] = testProb("fp2.txt")
    allprob[:,2] = testProb("fp3.txt")
    allprob[:,3] = testProb("fp6.txt")
    allprob[:,4] = testProb("fp7.txt")
    
    estimate = np.argmax(allprob,axis=1)
    print(estimate)
    for i in range(0,5) :
        if i==0:
            error[i] = len(np.where(estimate[:count[i]]!=i)[0])
        else :
            error[i] = len(np.where(estimate[count[i-1]:count[i]]!=i)[0])

    print(error)
    acc = 1-(sum(error)/count[-1])
    
    return acc

In [24]:
acc = accModel(count.astype(int))
print(acc)

[0 0 2 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 1 4 4 4 4 4 4]
[1. 1. 0. 0. 1.]
0.95


**Continuous HMMs**

In [30]:

!cp -avr /content/gdrive/My\ Drive/one.seq.hmm /root
!cp -avr /content/gdrive/My\ Drive/two.seq.hmm /root
!cp -avr /content/gdrive/My\ Drive/three.seq.hmm /root
!cp -avr /content/gdrive/My\ Drive/six.seq.hmm /root
!cp -avr /content/gdrive/My\ Drive/seven.seq.hmm /root
#!cp -avr /content/gdrive/My\ Drive/12.hmm /root
!ls

'/content/gdrive/My Drive/one.seq.hmm' -> '/root/one.seq.hmm'
'/content/gdrive/My Drive/two.seq.hmm' -> '/root/two.seq.hmm'
'/content/gdrive/My Drive/three.seq.hmm' -> '/root/three.seq.hmm'
'/content/gdrive/My Drive/six.seq.hmm' -> '/root/six.seq.hmm'
'/content/gdrive/My Drive/seven.seq.hmm' -> '/root/seven.seq.hmm'
1	6	   DHMM.test  fp6.txt	   seven.seq	  three.seq
12.hmm	7	   fp1.txt    fp7.txt	   seven.seq.hmm  three.seq.hmm
2	ctsHMM	   fp2.txt    one.seq	   six.seq	  two.seq
3	ctsModels  fp3.txt    one.seq.hmm  six.seq.hmm	  two.seq.hmm


In [49]:
                                    #Concatanate 2-digit models
def filecombine(file1,file2,fileNew,folder="ctsModels",Nstate="6",p=0.9,check=[]) :
                                    #Read file1 - on top
    data1 = np.genfromtxt(file1,delimiter='\n',dtype=str)
                                    #Change the number of states
    headers = np.genfromtxt(data1[:2],delimiter=' ',dtype=str)
    headers[0,1] = Nstate
    headers = list(headers)
    headers.append("\n")
                                    #Read the initial probabilities
    probs1 = np.genfromtxt(data1[2:],delimiter='\t',dtype=str)
    #probs1 = probs1[:,:-1]
    probs1 = probs1.astype("float")
    probs1[-2][0] = 1-p
    probs1[-1][0] = p
                                    #Read file2 - bottom
    data2 = np.genfromtxt(file2,delimiter='\n',dtype=str)
                                    #Read the initial probabilities
    probs2 = np.genfromtxt(data2[2:],delimiter='\t',dtype=str)
    #probs2 = probs2[:,:-1]
    probs2 = probs2.astype("float")
    print(shape(probs1),shape(probs2))
                                    #Change the last state transition probabilities
    probs = np.concatenate((probs1,probs2))
    
                                    #Write to file
    with open(fileNew,"w") as f:
        f.write("\n".join(" ".join(map(str, line)) for line in headers))

    f = open(fileNew,'ab')
    np.savetxt(f,probs,delimiter='\t', fmt='%f')
    !mv $fileNew /root/$folder
    f.close()

    return None

In [45]:
def digit2model(files=None,folder="ctsModels") :
    if files is None :
        files = ["one.seq.hmm","two.seq.hmm","three.seq.hmm","six.seq.hmm","seven.seq.hmm"]
    opfiles = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
    opfiles = opfiles.astype("str")
    for i in range(0,5) :
        for j in range(0,5) :
            filecombine(files[i],files[j],opfiles[int(5*i+j)]+".hmm")

    return None

In [ ]:
!mkdir ctsModels
digit2model()
# model2digit = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
# model2digit = model2digit.astype("str")
# for i in range(20,25) :
#     files.download(model2digit[i]+".hmm")

In [ ]:
!ls
%cd /root/ctsModels/
#!cp *.hmm /root
!ls
%cd

In [70]:
def digit3model(files1=None,files2=None,folder="ctsModels") :
    if files1 is None :
        files1 = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
        files1 = files1.astype("str")
    if files2 is None :
        files2 = array([1,2,3,6,7])
        files2 = files2.astype("str")
        files = ["one.seq.hmm","two.seq.hmm","three.seq.hmm","six.seq.hmm","seven.seq.hmm"]
    opfiles = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
    opfiles = opfiles.astype("str")
    for i in range(0,25) :
        for j in range(0,5) :
            filecombine(files1[i]+".hmm",files[j],files1[i]+files2[j]+".hmm",Nstate="9")

    return None

In [ ]:
digit3model()

In [73]:
!zip -r /root/ctsModels.zip /root/ctsModels

files.download("/root/ctsModels.zip")

  adding: root/ctsModels/ (stored 0%)
  adding: root/ctsModels/613.hmm (deflated 77%)
  adding: root/ctsModels/12.hmm (deflated 77%)
  adding: root/ctsModels/137.hmm (deflated 78%)
  adding: root/ctsModels/662.hmm (deflated 83%)
  adding: root/ctsModels/766.hmm (deflated 83%)
  adding: root/ctsModels/162.hmm (deflated 78%)
  adding: root/ctsModels/313.hmm (deflated 83%)
  adding: root/ctsModels/61.hmm (deflated 76%)
  adding: root/ctsModels/672.hmm (deflated 77%)
  adding: root/ctsModels/716.hmm (deflated 77%)
  adding: root/ctsModels/767.hmm (deflated 82%)
  adding: root/ctsModels/76.hmm (deflated 75%)
  adding: root/ctsModels/627.hmm (deflated 77%)
  adding: root/ctsModels/176.hmm (deflated 78%)
  adding: root/ctsModels/316.hmm (deflated 77%)
  adding: root/ctsModels/317.hmm (deflated 78%)
  adding: root/ctsModels/617.hmm (deflated 78%)
  adding: root/ctsModels/621.hmm (deflated 78%)
  adding: root/ctsModels/731.hmm (deflated 77%)
  adding: root/ctsModels/362.hmm (deflated 77%)
  add

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
a = array([11,12,13])
a = a.astype("str")
a[1]=="12"

True

In [ ]:
#filecombine("one.seq.hmm","two.seq.hmm","check.txt")
filecombine("12.hmm","three.seq.hmm","123.hmm")

(12, 17) (6, 17)


In [ ]:
files.download("123.hmm")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
                                    #Give the file with sequences for cts. dev data
def ctsdevSeq(trainKM,fileWord="check.txt") :
    seq = []   
                                    #Go through required folders
    for subdir, dirs, files in os.walk("ctsHMM"):
        print(subdir)
        dirname = subdir.split(os.path.sep)[-1]
        print('Directory:', dirname)
        if dirname == "dev":
            for f in files:
                if f.endswith(".mfcc"):
                                    #Copy file (mfcc) to root and perform K-means
                    !cp -avr /root/ctsHMM/dev/$f /root
                    label = testKmeans(f,trainKM)
                    seq.append(label)
                    !rm $f

    seq = array(seq)
    
    seqDwnld(seq,fileWord)

    return None

In [80]:
                                    #Get the test sequence
ctsdevSeq(cenKM,"ctsHMM.dev")

ctsHMM
Directory: ctsHMM
ctsHMM/test
Directory: test
ctsHMM/dev
Directory: dev
'/root/ctsHMM/dev/262.mfcc' -> '/root/262.mfcc'
'/root/ctsHMM/dev/662.mfcc' -> '/root/662.mfcc'
'/root/ctsHMM/dev/66.mfcc' -> '/root/66.mfcc'
'/root/ctsHMM/dev/711.mfcc' -> '/root/711.mfcc'
'/root/ctsHMM/dev/12.mfcc' -> '/root/12.mfcc'
'/root/ctsHMM/dev/233.mfcc' -> '/root/233.mfcc'
'/root/ctsHMM/dev/217.mfcc' -> '/root/217.mfcc'
'/root/ctsHMM/dev/121.mfcc' -> '/root/121.mfcc'
'/root/ctsHMM/dev/622.mfcc' -> '/root/622.mfcc'
'/root/ctsHMM/dev/61.mfcc' -> '/root/61.mfcc'
'/root/ctsHMM/dev/11.mfcc' -> '/root/11.mfcc'
'/root/ctsHMM/dev/13.mfcc' -> '/root/13.mfcc'
'/root/ctsHMM/dev/122.mfcc' -> '/root/122.mfcc'
'/root/ctsHMM/dev/22.mfcc' -> '/root/22.mfcc'
'/root/ctsHMM/dev/111.mfcc' -> '/root/111.mfcc'
'/root/ctsHMM/dev/77.mfcc' -> '/root/77.mfcc'
'/root/ctsHMM/dev/277.mfcc' -> '/root/277.mfcc'
'/root/ctsHMM/dev/212.mfcc' -> '/root/212.mfcc'
'/root/ctsHMM/dev/16.mfcc' -> '/root/16.mfcc'
'/root/ctsHMM/dev/331.mfc

In [81]:
                                    #Download the test sequence files
files.download("ctsHMM.dev")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [83]:
!ls
%cd ctsHMM/dev
!ls
%cd

1	22.hmm	36.hmm	7	    ctsModels	   one.seq	  two.seq
11.hmm	23.hmm	37.hmm	71.hmm	    ctsModels.zip  one.seq.hmm	  two.seq.hmm
12.hmm	26.hmm	6	72.hmm	    DHMM.test	   seven.seq
13.hmm	27.hmm	61.hmm	73.hmm	    fp1.txt	   seven.seq.hmm
16.hmm	3	62.hmm	76.hmm	    fp2.txt	   six.seq
17.hmm	31.hmm	63.hmm	77.hmm	    fp3.txt	   six.seq.hmm
2	32.hmm	66.hmm	ctsHMM	    fp6.txt	   three.seq
21.hmm	33.hmm	67.hmm	ctsHMM.dev  fp7.txt	   three.seq.hmm
/root/ctsHMM/dev
111.mfcc  122.mfcc  16.mfcc   21.mfcc	262.mfcc  322.mfcc  662.mfcc  77.mfcc
116.mfcc  12.mfcc   172.mfcc  22.mfcc	277.mfcc  331.mfcc  66.mfcc
11.mfcc   132.mfcc  212.mfcc  233.mfcc	311.mfcc  61.mfcc   711.mfcc
121.mfcc  13.mfcc   217.mfcc  23.mfcc	31.mfcc   622.mfcc  71.mfcc
/root


In [90]:
dump(cenKM, open('cenKM.pkl', 'wb'))

In [91]:
cenCheck = load(open('cenKM.pkl', 'rb'))

In [99]:
                                    #Test file
!cp -avr /root/7/dev/ra_7.mfcc /root
filex = "ra_7.mfcc"
label,ft = fileTofeature(filex)
true = cenKM.predict(ft)
pklcheck = cenCheck.predict(ft)
print(true==pklcheck)

'/root/7/dev/ra_7.mfcc' -> '/root/ra_7.mfcc'
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]


In [ ]:
a = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
b = array([1,2,3,6,7])
a = a.astype("str")
b = b.astype("str")
c = np.zeros(125)
for i in range(0,25) :
    for j in range(0,5) :
        c[int(5*i+j)] = a[i]+b[j]

c = c.astype("int")
c = c.astype("str")
c = list(c)
a = list(a)
d = a+c
print(c)
print(a)
print(d)

['111', '112', '113', '116', '117', '121', '122', '123', '126', '127', '131', '132', '133', '136', '137', '161', '162', '163', '166', '167', '171', '172', '173', '176', '177', '211', '212', '213', '216', '217', '221', '222', '223', '226', '227', '231', '232', '233', '236', '237', '261', '262', '263', '266', '267', '271', '272', '273', '276', '277', '311', '312', '313', '316', '317', '321', '322', '323', '326', '327', '331', '332', '333', '336', '337', '361', '362', '363', '366', '367', '371', '372', '373', '376', '377', '611', '612', '613', '616', '617', '621', '622', '623', '626', '627', '631', '632', '633', '636', '637', '661', '662', '663', '666', '667', '671', '672', '673', '676', '677', '711', '712', '713', '716', '717', '721', '722', '723', '726', '727', '731', '732', '733', '736', '737', '761', '762', '763', '766', '767', '771', '772', '773', '776', '777']
['11', '12', '13', '16', '17', '21', '22', '23', '26', '27', '31', '32', '33', '36', '37', '61', '62', '63', '66', '67', '71

In [ ]:
list(cenKM.cluster_centers_)

In [88]:
check = array([array([-1.66948264e+00, -2.96174904e+00, -1.41978217e+00,  9.35223380e-01,
        -2.01945790e+00, -2.91612342e-01, -1.10957615e+00, -8.16972554e-01,
        -2.23668776e-01, -1.60549106e+00, -5.35263855e-01, -1.30204218e+00,
        -8.40138393e-01, -1.87181569e-01, -9.37894650e-01, -3.23093229e-01,
        -1.03882455e+00, -6.70679960e-01, -4.64184149e-01,  9.79252846e-02,
         2.50065029e-02, -2.08390099e-02,  2.92831255e-02, -2.06034299e-02,
         1.67657629e-02,  1.03762488e-03, -4.21936932e-05,  1.65299388e-02,
        -4.81788948e-03,  1.31192493e-02, -4.86370833e-03,  1.99328523e-03,
         7.21018080e-03, -9.90213795e-03,  8.35632350e-03, -2.96198913e-03,
         3.98993289e-03,  6.29460375e-03]),
 array([ 1.60296543, -0.05448187, -2.07454556, -1.99068772,  2.78819727,
         2.10391858,  3.34141639,  2.92309178,  2.44879755,  3.74164573,
         1.67949942,  1.44937612,  0.61013968, -0.23785792,  0.81138365,
         0.55186602,  1.21846269,  1.07396922,  0.96587281,  0.22175469,
         0.08391972, -0.07559218, -0.21446493,  0.0946653 , -0.01968363,
         0.00695253, -0.08066623, -0.05758038,  0.18628622,  0.11832907,
         0.17460868,  0.12320789,  0.03901573,  0.09333297, -0.00469206,
         0.0041072 , -0.0088512 ,  0.00652068]),
 array([-2.18850124e+01, -1.25164659e+00,  4.15915391e+00, -5.26298168e+00,
         2.81134143e+00, -3.86389857e+00,  3.10397526e-01,  4.28807820e-01,
        -2.67901736e+00,  1.30446590e+00, -2.42788288e+00,  5.32930312e-01,
        -4.18349440e-01, -1.60652194e+00,  1.10691929e+00, -1.83408148e+00,
         6.60067567e-01, -5.42725135e-01, -1.11607320e+00, -1.16661157e-01,
        -1.30717561e-01, -2.98008886e-02,  2.97284050e-02, -1.17809799e-01,
        -2.18312599e-02, -7.08199222e-02, -6.05965781e-02, -3.95924924e-02,
        -1.07892576e-01, -5.59425609e-02, -6.93003677e-02, -2.20691064e-02,
        -1.48783310e-02, -6.37984579e-02, -4.36928331e-02, -9.22633688e-02,
        -4.20704002e-02, -1.19836655e-02]),
 array([ 6.3666496 ,  4.52841695, -1.34474259, -2.7395294 ,  4.34703103,
         0.76033259,  0.46592251,  0.5749837 ,  0.8761906 ,  3.0825035 ,
         1.15123678,  2.28990445,  1.4753111 ,  0.40250686,  1.72029824,
         0.6660268 ,  1.35484744,  0.72813217,  0.48857534, -0.37169391,
         0.03255512,  0.16300808, -0.08520941,  0.03132444, -0.06950836,
        -0.04210587, -0.03914499, -0.08304792, -0.03481417, -0.07910017,
        -0.00993265, -0.01312571,  0.01205243,  0.07484959, -0.006983  ,
         0.01167578, -0.02115743, -0.03812324]),
 array([ 6.62869371,  2.40609027,  1.26175186,  1.73991427,  0.27597016,
         0.80208556,  0.85046345,  1.30977106,  2.20703256,  2.05906957,
         2.0201844 ,  1.28376991,  0.94556631,  0.79968461,  0.67522684,
         1.28758009,  1.18219286,  1.03520586,  0.8291044 ,  0.16574204,
         0.25973215,  0.10230428, -0.09107877,  0.22818536,  0.05467388,
         0.09596279,  0.02921341, -0.02018844,  0.13450462,  0.03480599,
         0.10513473,  0.03840598, -0.03314963,  0.04410179, -0.00701333,
         0.09076805,  0.05852722,  0.01601057]),
 array([-4.86268665e+00, -3.38415822e+00, -1.12869905e+00,  2.96792522e-02,
        -1.71954594e+00, -1.04881815e+00, -1.17785835e+00, -7.44718503e-01,
        -5.05813301e-01, -1.23951134e+00, -6.84976212e-01, -9.96658494e-01,
        -7.47161938e-01, -4.13705271e-01, -7.99496474e-01, -4.68778661e-01,
        -7.28620989e-01, -6.12040335e-01, -5.27101804e-01,  9.42733590e-02,
         1.38921370e-03, -2.73495451e-02,  2.95593302e-02, -4.21226062e-02,
         1.88097333e-02, -1.57234112e-02, -2.09545025e-02,  7.62002484e-03,
        -2.10301896e-02,  1.00588905e-02, -1.48076409e-02, -4.30243954e-03,
         1.64563726e-02, -3.43253442e-03,  1.33901358e-02, -1.12026566e-02,
        -4.65011001e-03,  2.32090474e-03]),
 array([ 4.43668458e+00,  5.26617986e+00,  2.01852628e+00, -1.96413375e+00,
         3.29879992e+00,  1.55383704e+00,  1.49345132e+00,  2.51936922e-01,
        -1.04625674e+00, -7.36104720e-02, -1.81946140e+00,  8.28703763e-02,
         3.62070617e-01, -7.97801736e-02,  1.01408798e+00, -3.32323359e-01,
         4.08835556e-01,  4.59994087e-01,  4.93327592e-01,  2.72170603e-01,
         8.26542053e-02,  2.19023368e-02,  2.17663590e-01,  1.02568521e-01,
         2.66902631e-01,  1.59991436e-01,  7.02028019e-02,  1.07080273e-01,
         2.70671644e-03,  1.80439698e-02, -4.15440923e-02, -4.60225039e-02,
        -4.30679741e-02, -1.02013832e-01, -7.90719541e-02, -1.32138171e-01,
        -8.37471526e-02, -1.45149857e-02]),
 array([ 7.36551401e+00,  5.92678972e-01, -9.77812104e-01,  2.24992288e+00,
        -1.50937386e+00,  8.56327712e-01, -4.43743796e-01, -4.85978688e-01,
         3.69260268e-01, -1.22446979e+00,  3.56450467e-01, -6.09264973e-01,
        -8.85799532e-02,  3.89578241e-01, -7.13013175e-01,  3.63873273e-01,
        -5.82891409e-01, -4.55086021e-02,  2.45576618e-01,  1.44298337e-01,
         4.18707239e-03, -1.72652301e-03, -2.04117927e-03,  3.46162952e-03,
        -8.56938215e-03,  2.97904158e-02,  3.06631988e-02,  2.52450270e-02,
         4.54423598e-02,  4.03608543e-02,  5.11676756e-02,  2.80730867e-02,
         1.26292410e-03,  1.24812789e-02,  2.07916620e-02,  3.07297860e-02,
         1.75320406e-02,  1.19903151e-02]),
 array([-15.82226077,  -0.49355357,   3.03826067,  -4.49406527,
          2.19921692,  -3.06806279,   0.22486291,   0.49019274,
         -2.06819044,   0.94137069,  -1.90815147,   0.49304281,
         -0.26897943,  -1.2400846 ,   0.88417303,  -1.33077125,
          0.71706553,  -0.30664519,  -0.86024728,  -0.25298806,
         -0.10159397,  -0.03227674,  -0.02200442,  -0.0777086 ,
         -0.04979189,  -0.07434039,  -0.06270559,  -0.07753513,
         -0.11609255,  -0.07034707,  -0.08435035,  -0.06523331,
         -0.02308122,  -0.05031418,  -0.06584084,  -0.08288418,
         -0.06114697,  -0.03278938]),
 array([ 1.24869953e+01,  2.53960940e+00, -3.04389413e+00, -6.75919660e-01,
         3.03881284e+00,  1.39717763e+00,  6.81675963e-01,  6.76229883e-01,
         1.69893714e+00,  3.14585914e+00,  2.40366533e+00,  2.37763495e+00,
         1.51804328e+00,  5.10845902e-01,  1.11032924e+00,  8.81381122e-01,
         8.11476880e-01,  9.20114528e-01,  8.34546779e-01, -9.18353966e-02,
         1.68255507e-01,  1.37677583e-01, -1.61028710e-01,  2.10033205e-01,
        -4.36372646e-02,  1.71693674e-02, -1.00304611e-02, -1.08375322e-01,
         2.28592514e-02, -1.00176264e-01,  5.50006651e-02,  5.12710883e-02,
         6.30988014e-03,  1.21873161e-01,  3.94522713e-02,  1.05908235e-01,
         1.86907536e-02, -4.48152336e-02]),
 array([-1.09695337e+01, -2.67692007e+00,  1.05858184e+00, -3.19129954e+00,
         9.65290928e-01, -2.39685870e+00, -1.98793377e-01,  1.46874549e-01,
        -1.59272751e+00,  2.20889797e-01, -1.43904373e+00,  2.04411880e-01,
        -2.78113492e-01, -9.32619088e-01,  4.07715520e-01, -9.74078265e-01,
         3.72768061e-01, -2.62444743e-01, -7.17418275e-01,  7.27558134e-01,
        -1.64182031e-03, -1.48927284e-01,  2.69829053e-01, -1.95273300e-01,
         1.10241989e-01, -8.65698505e-02, -8.71436953e-02,  4.73232357e-02,
        -1.57772220e-01,  2.18327697e-02, -1.19832009e-01, -3.81723603e-02,
         4.05925796e-02, -8.59982745e-02,  5.58697748e-02, -9.89756674e-02,
        -4.18646837e-02,  1.34760639e-02]),
 array([ 9.44131925e+00,  9.00972410e+00,  3.88934969e+00,  1.58532370e+00,
         4.05786543e+00,  1.24360963e+00, -1.29004788e-01, -1.54791028e+00,
        -2.06226979e+00, -5.44926597e-01, -3.70050550e-01,  1.29269215e+00,
         1.39382846e+00,  1.40156556e+00,  2.23374996e+00,  1.20685636e+00,
         1.25274986e+00,  8.64893281e-01,  8.18038742e-01, -1.69086945e-01,
        -1.12912507e-01,  1.75233532e-01,  2.43269611e-01, -1.05533315e-01,
         1.60172948e-02,  6.62314408e-02,  1.59361008e-01,  1.32025326e-01,
        -1.18005262e-01, -1.38143458e-01, -1.84859772e-01, -1.13936468e-01,
        -5.85712469e-02, -1.14846402e-01, -6.50839176e-03,  2.71460995e-02,
         4.41773113e-02,  2.66472711e-02]),
 array([ 1.02916815e+01,  4.35300850e+00,  7.87593486e-01, -2.12819712e-01,
         6.08287477e-01,  1.59337536e+00,  1.58648298e+00,  1.73725490e-01,
         3.81782295e-01,  1.41342416e+00,  1.16033378e+00,  1.43715350e+00,
         1.43034948e+00,  7.93113570e-01,  6.51104608e-01,  7.65199079e-01,
         1.13429767e+00,  1.13978705e+00,  9.57172091e-01, -1.92097301e-01,
         2.82829529e-03,  2.77052595e-02, -1.81685873e-01,  8.02078672e-02,
        -5.85850912e-02,  2.08260105e-02, -4.14364182e-02, -1.10876808e-01,
         1.45471327e-02, -9.16246064e-02,  1.06640532e-02,  1.25513136e-03,
        -4.02512234e-02,  3.63554610e-02, -2.00862718e-02,  4.64901996e-02,
        -7.46243361e-03, -3.91455076e-02]),
 array([ 1.37147198, -0.68217445, -0.95738027,  0.00252958, -2.20939695,
        -0.89652681, -1.06787835, -0.66647873, -0.29354279, -1.20928408,
        -0.36670019, -0.79900944, -0.44386481, -0.06355981, -0.6822388 ,
        -0.2323087 , -0.63068759, -0.39011112, -0.29821095,  0.22857095,
         0.05798298, -0.016451  ,  0.01046802,  0.00351844,  0.03475194,
         0.02826217,  0.04902418,  0.07825003,  0.06557677,  0.06095443,
         0.03776914,  0.03454764,  0.02799353,  0.01751525,  0.03351143,
         0.03074167,  0.03579572,  0.02797731]),
 array([ 3.20376513e+00, -1.37788664e+00, -1.63144430e+00,  1.87804441e+00,
        -1.49398000e+00,  9.92461297e-01, -4.46049036e-01, -4.50372328e-01,
         4.78943817e-01, -1.20470325e+00,  1.81127623e-01, -9.41803967e-01,
        -4.00810717e-01,  2.78148600e-01, -7.81975801e-01,  1.93273059e-01,
        -7.86336211e-01, -2.72549739e-01,  9.95165688e-02,  1.29784171e-01,
         1.04448789e-02, -3.92998071e-02,  2.75800146e-02, -1.23105489e-03,
         3.69106636e-02,  1.37171042e-02,  1.54520322e-02,  4.25906896e-02,
         2.33791639e-02,  3.69032038e-02,  1.49487521e-02,  1.28154899e-02,
         1.78520649e-02,  4.70177623e-03,  1.73565744e-02,  3.88635567e-03,
         1.29457188e-02,  1.60069581e-02]),
 array([ 2.45262326,  0.89853674,  1.02033155,  1.88849329, -0.38393166,
         0.47011527,  0.10969051,  0.42770276,  0.69770794, -0.15651322,
         0.5943473 ,  0.08276972,  0.2595424 ,  0.50342292,  0.02098138,
         0.55051351,  0.11381147,  0.27655176,  0.39747548,  0.24505142,
         0.13405623,  0.03352473,  0.05810437,  0.0747889 ,  0.06436558,
         0.07213333,  0.07879869,  0.09070503,  0.10070297,  0.08163015,
         0.05445657,  0.03527296,  0.01742672,  0.02157693,  0.04380864,
         0.05054661,  0.04800021,  0.04771406]),
 array([-6.17363120e+00,  4.76228714e-03,  2.00036492e+00, -2.03395542e+00,
         1.50395996e+00, -1.46630501e+00,  7.32997010e-01,  1.05459450e+00,
        -5.25290046e-01,  1.03823519e+00, -4.99251476e-01,  9.19576785e-01,
         2.97041846e-01, -4.24890663e-01,  8.40466459e-01, -2.52127806e-01,
         1.01850355e+00,  3.68813686e-01, -1.58855538e-01, -2.10126239e+00,
        -6.92647788e-01,  1.41989049e-02, -3.10590237e-01, -8.43803003e-02,
        -3.41236643e-01, -1.29728375e-01, -5.94792512e-02, -2.83853631e-01,
        -2.22406797e-01, -3.20205625e-01, -2.13040896e-01, -2.08202016e-01,
        -1.95073182e-01, -1.26406291e-01, -2.55407750e-01, -1.46369221e-01,
        -1.63849999e-01, -1.65829592e-01]),
 array([-1.71469646e+00, -1.19328786e+00, -2.30970103e-02,  1.06338109e+00,
        -9.85214008e-01, -1.67835502e-01, -4.86624734e-01, -1.77971727e-01,
         5.76105325e-02, -8.39672987e-01, -1.69705966e-01, -5.94132718e-01,
        -3.73187784e-01,  7.68926495e-03, -4.76211607e-01, -1.22183671e-01,
        -4.50385250e-01, -2.74746503e-01, -1.99336112e-01,  7.95538510e-02,
         1.13326596e-03, -2.84480918e-02,  6.87787005e-03, -3.01890234e-02,
        -1.26589479e-03, -1.40564907e-02, -8.48673569e-03,  3.02416668e-03,
        -1.51112601e-02,  3.17645849e-03, -9.86397825e-03, -5.44419941e-03,
        -1.33500296e-03, -1.10468225e-02,  4.82765582e-03, -6.67139350e-03,
         3.97074883e-04,  3.30077802e-03]),
 array([ 3.24468401e+00,  4.21868940e+00,  1.70108163e+00, -2.35004766e+00,
         2.00701409e+00,  8.08284589e-01,  2.16484212e+00,  7.09539153e-01,
        -1.18723096e-01,  2.89672435e+00,  1.70765703e+00,  2.33667215e+00,
         1.40487993e+00,  5.77400186e-01,  1.70061848e+00,  8.95382418e-01,
         1.84239483e+00,  9.20715414e-01,  5.68274436e-01, -8.72937503e-01,
        -2.40505187e-01,  8.00351367e-02, -1.70334837e-01, -2.20396799e-03,
        -2.17282764e-01, -6.49577894e-02,  1.97225011e-02, -1.17106918e-01,
        -7.93968323e-02, -1.46768334e-01, -3.00564451e-02, -3.33021339e-02,
        -4.91527781e-02,  2.66981418e-02, -7.58488820e-02, -3.35432949e-02,
        -4.73329534e-02, -6.89383794e-02]),
 array([ 5.21261907,  5.25869353,  3.03698388,  1.96993078,  3.71388161,
         3.07623123,  2.58079039,  1.49904019,  1.20482147,  1.78877837,
         1.05567151,  1.23810239,  0.70420309,  0.10549732,  0.44995379,
         0.14027976,  0.4806194 ,  0.67205142,  0.76337985, -0.24750131,
        -0.13073494, -0.07594787, -0.0059635 , -0.04938843, -0.10466461,
        -0.05658581, -0.02794859,  0.00791283,  0.09492037,  0.12729445,
         0.08618217,  0.04985597,  0.03958093,  0.02595551,  0.05507757,
         0.09725182,  0.06697181,  0.04549541])])

In [97]:
files.download("/root/cenKM.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>